## Importing libraries, dataset and preprocessing the dataset

In [1]:
# Import necessary libraries
import random
import json
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import pickle


# Load NLTK resources
nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents from JSON file
with open('output.json') as json_file:
    intents = json.load(json_file)

# Initialize empty lists
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Preprocess data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)  # Tokenize words
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize words and remove duplicates
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
training = []
output_empty = [0] * len(classes)

# Prepare input-output pairs
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# Shuffle training data
random.shuffle(training)

# Convert training data to numpy arrays
train_x = np.array([row[0] for row in training])
train_y = np.array([row[1] for row in training])


## Defining the model,training on prepared dataset and saving the model(localy)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Create sequential model
model = Sequential()

# Add dense layers
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
optimizer = Adam(lr=0.0001)  # Use Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train model
hist = model.fit(train_x, train_y, epochs=100, batch_size=10, verbose=1)

# Save model
model.save('chatbotmodel.h5')

print('Training Done')


Epoch 1/100



49/49 [==============================] - 1s 2ms/step - loss: 4.6836 - accuracy: 0.0246
Epoch 2/100
49/49 [==============================] - 0s 2ms/step - loss: 4.5570 - accuracy: 0.0656
Epoch 3/100
49/49 [==============================] - 0s 1ms/step - loss: 4.4089 - accuracy: 0.0635
Epoch 4/100
49/49 [==============================] - 0s 1ms/step - loss: 4.1990 - accuracy: 0.0574
Epoch 5/100
49/49 [==============================] - 0s 2ms/step - loss: 4.0440 - accuracy: 0.0984
Epoch 6/100
49/49 [==============================] - 0s 2ms/step - loss: 3.8871 - accuracy: 0.1270
Epoch 7/100
49/49 [==============================] - 0s 1ms/step - loss: 3.6810 - accuracy: 0.1475
Epoch 8/100
49/49 [==============================] - 0s 2ms/step - loss: 3.5046 - accuracy: 0.1824
Epoch 9/100
49/49 [==============================] - 0s 1ms/step - loss: 3.3993 - accuracy: 0.1701
Epoch 10/100
49/49 [==============================] - 0s 1ms/step - loss: 3.1989 - accuracy: 0.2377
Epoch 11/100
49/49 [=

c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
# Save model
model.save('chatbotmodel.h5')

## Running on user inputs

In [5]:
import nltk
import random
import numpy as np
import json
import pickle
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
lemmatizer=WordNetLemmatizer()

with open('mental_health_intents.json') as json_file:
    intents = json.load(json_file)

words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbotmodel.h5')

def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i]=1
  return np.array(bag)

def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25
  results=[[i,r] for i,r in enumerate(res) if r> ERROR_THRESHOLD]

  results.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent': classes[r[0]],'probability':str(r[1])})
  return return_list

def get_response(intents_list,intents_json):
  if len(intents_list)==0:
    return "I am still learning new things :)"
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

print("GO! BOT IS RUNNING")
message='hello'
while message!='exit':
  message=input("user >> ")
  ints=predict_class(message)
  print("predicted class: ", ints)
  res=get_response(ints,intents)
  print("bot >> "+ res)

GO! BOT IS RUNNING
1/1 [==============================] - 0s 176ms/step
predicted class:  [{'intent': 'greeting', 'probability': '0.9998036'}]
bot >> Great to see you. How do you feel currently?
1/1 [==============================] - 0s 34ms/step
predicted class:  [{'intent': 'sad', 'probability': '0.9989692'}]
bot >> Why do you think you feel this way?
1/1 [==============================] - 0s 28ms/step
predicted class:  [{'intent': 'happy', 'probability': '0.99730587'}]
bot >> That's geat to hear. I'm glad you're feeling this way.
1/1 [==============================] - 0s 30ms/step
predicted class:  [{'intent': 'greeting', 'probability': '0.31337437'}, {'intent': 'casual', 'probability': '0.28924954'}]
bot >> Hi there. What brings you here today?
1/1 [==============================] - 0s 26ms/step
predicted class:  [{'intent': 'casual', 'probability': '0.3165822'}]
bot >> Can you elaborate on that?
